Librerías del Proyecto

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from nltk.tokenize import word_tokenize
import os
import re
import string
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')
 
# Importing libraries necessary for Model Building and Training
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
 
import warnings
warnings.filterwarnings('ignore')

import mailbox
from email import policy
from email.parser import BytesParser

In [ ]:
archivo_correo = mailbox.mbox(os.getcwd() + r'\easy_ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e')

with open(os.getcwd() + r'\easy_ham\0001.ea7e79d3153e7469e7a9c3e0af6a357e', 'rb') as fp:
    archivo_correo_2 = BytesParser(policy=policy.default).parse(fp)

archivo_correo_2.get_payload()


In [ ]:

for mensaje in archivo_correo:
    print(mensaje.is_multipart())
    cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
cuerpo

Abriendo las carpetas de spam y no spam:

In [ ]:
os.getcwd()

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
            if c == '<' and not quote:
                tag = True
            elif c == '>' and not quote:
                tag = False
            elif (c == '"' or c == "'") and tag:
                quote = not quote
            elif not tag:
                out = out + c

    return out

In [ ]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

In [ ]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://S+|www.S+')
    return url_pattern.sub(r'', text)

In [ ]:
def remove_numbers(text):
    return ''.join(c for c in text if not c.isdigit())

In [ ]:
def remove_stopwords(text):
    stop_words = stopwords.words('english')
 
    imp_words = []
 
    # Storing the important words
    for word in str(text).split():
        word = word.lower()
 
        if word not in stop_words:
            imp_words.append(word)
 
    output = " ".join(imp_words)
 
    return output

In [ ]:
def email_parser(mbox, carp):
    message = {}
    for mensaje in mbox:
        cuerpo = ""
        if mensaje.is_multipart():
            # Si el correo tiene varias partes, buscar la parte de texto/html
            # message['isMultipart'] = True
            for parte in mensaje.get_payload():
                if parte.get_content_type() == 'text/plain':
                    cuerpo = parte.get_payload(decode=True).decode(errors='replace')
                    #cuerpo += cuerpo.translate(str.maketrans('', '', string.punctuation))
                if parte.get_content_type() == 'text/html':
                    cuerpo = parte.get_payload(decode=True).decode(errors='replace')
                    #cuerpo += remove_html_markup(cuerpo)
        else:
            # message['isMultipart'] = False
            # Si el correo es de una sola parte, obtener el cuerpo directamente
            if mensaje.get_content_type() == 'text/plain':
                char_set = mensaje.get_charset()
                cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
                #cuerpo = cuerpo.translate(str.maketrans('', '', string.punctuation))
            if mensaje.get_content_type() == 'text/html':
                char_set = mensaje.get_charset()
                cuerpo = mensaje.get_payload(decode=True).decode(errors='replace')
                #cuerpo = remove_html_markup(cuerpo)
        message['message'] = cuerpo
        if carp == 'easy_ham':
            message['type'] = 0
        else:
            message['type'] = 1

        return message

In [ ]:
carpeta = ["spam", "easy_ham"]  # Ruta de la carpeta que deseas leer
message_list = []
for carp in carpeta:
    # Obtener la lista de archivos en la carpeta
    lista_archivos = [f'{os.path.join(os.getcwd(), carp)}\\{x}' for x in os.listdir(os.path.join(os.getcwd(), carp))]
    #print(lista_archivos)
    # Leer cada archivo de la lista
    for archivo in lista_archivos:
        if os.path.isfile(archivo):  # Verificar que sea un archivo
            # Abrir el dataset de correos
            mbox = mailbox.mbox(archivo)
            # Recorrer cada correo en el dataset
            message = email_parser(mbox, carp)
            if message == None:
                print(archivo)
            else:
                
                message_list.append(message)

In [ ]:
for x in message_list:
    x["message"] = re.sub(r"[^\x00-\x7F]+", "", str(x["message"]))
    x["message"] = x["message"].replace("\n", " ").replace("\t", " ").replace("\\u", " ").replace("\r", " ").lower()
    x["message"] = cleanhtml(x["message"])
    x["message"] = remove_html_markup(x["message"])
    x["message"] = remove_urls(x["message"])
    x["message"] = remove_punctuation(x["message"])
    x["message"] = remove_numbers(x["message"])
    x["message"] = remove_stopwords(x["message"])

In [ ]:
with open('messages.csv', 'w') as f:
    f.write(f'isSpam,message\n')
    for x in message_list:
        f.write(f'{x["type"]},{x["message"]}\n')

In [ ]:
data = pd.read_csv("messages.csv")
data.message = data.message.astype('str')
data.head()

In [ ]:
data.shape

In [ ]:
sns.countplot(x='isSpam', data=data)

In [ ]:
# Downsampling to balance the dataset
ham_msg = data[data.isSpam == 0]
spam_msg = data[data.isSpam == 1]
ham_msg = ham_msg.sample(n=len(spam_msg))
 
# Plotting the counts of down sampled dataset
balanced_data = ham_msg._append(spam_msg).reset_index(drop=True)
plt.figure(figsize=(8, 6))
sns.countplot(data = balanced_data, x='isSpam')
plt.title('Distribution of Ham and Spam email messages after downsampling')
plt.xlabel('Message types')

In [ ]:

#train test split
train_X, test_X, train_Y, test_Y = train_test_split(balanced_data['message'],
                                                    balanced_data['isSpam'],
                                                    test_size = 0.2,
                                                    random_state = 42)

In [ ]:

def plot_word_cloud(data, typ):
    email_corpus = " ".join(data['message'])
 
    plt.figure(figsize=(7, 7))
 
    wc = WordCloud(background_color='black',
                   max_words=100,
                   width=800,
                   height=400,
                   collocations=False).generate(email_corpus)
 
    plt.imshow(wc, interpolation='bilinear')
    plt.title(f'WordCloud for {typ} emails', fontsize=15)
    plt.axis('off')
    plt.show()
 
plot_word_cloud(balanced_data[balanced_data['isSpam'] == 0], typ='Non-Spam')
plot_word_cloud(balanced_data[balanced_data['isSpam'] == 1], typ='Spam')

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_X)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_X)
test_sequences = tokenizer.texts_to_sequences(test_X)

# Pad sequences to have the same length
max_len = 100 # maximum sequence length
train_sequences = pad_sequences(train_sequences,
								maxlen=max_len, 
								padding='post', 
								truncating='post')
test_sequences = pad_sequences(test_sequences, 
							maxlen=max_len, 
							padding='post', 
							truncating='post')


In [ ]:
# Build the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1,
                                    output_dim=32, 
                                    input_length=max_len))
model.add(tf.keras.layers.LSTM(16))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
 
# Print the model summary
model.summary()

In [ ]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'],
              optimizer = 'adam')

In [ ]:
es = EarlyStopping(patience=3,
                   monitor = 'val_accuracy',
                   restore_best_weights = True)
 
lr = ReduceLROnPlateau(patience = 2,
                       monitor = 'val_loss',
                       factor = 0.5,
                       verbose = 0)

In [ ]:
# Train the model
history = model.fit(train_sequences, train_Y,
                    validation_data=(test_sequences, test_Y),
                    epochs=20, 
                    batch_size=32,
                    callbacks = [lr, es]
                   )

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_sequences, test_Y)
print('Test Loss :',test_loss)
print('Test Accuracy :',test_accuracy)

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Leer el archivo CSV con los datos preparados
data = pd.read_csv("messages.csv")
data.message = data.message.astype('str')
# Inicializar contadores para spam y no spam
spam_count = 0
non_spam_count = 0

# Función para predecir si un correo es spam o no
def predict_spam(message_text):
    # Convertir el mensaje en secuencias y hacer la predicción
    mensaje_secuencia = tokenizer.texts_to_sequences([message_text])
    mensaje_secuencia = pad_sequences(mensaje_secuencia, maxlen=max_len, padding='post', truncating='post')
    prediction = model.predict(np.array(mensaje_secuencia))
    print(prediction)
    return prediction[0][0] >= 0.5  # Probabilidad de corte 0.5

# Realizar predicciones y contar la cantidad de correos clasificados como spam y no spam
for index, row in data.iterrows():
    is_spam = predict_spam(row['message'])
    print(f"Correo: {index} - Es spam: {is_spam}")

    # Incrementar los contadores según la predicción
    if is_spam:
        spam_count += 1
    else:
        non_spam_count += 1

# Imprimir la cantidad de correos clasificados como spam y no spam
print(f"\nCantidad de correos clasificados como spam: {spam_count}")
print(f"Cantidad de correos clasificados como no spam: {non_spam_count}")